In [1]:
import cv2
import numpy as np
from collections import OrderedDict
from imutils import face_utils
import dlib

In [2]:
filename = 0

# initializing landmark detection and face region detection
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')

In [80]:
FACIAL_LANDMARKS_IDXS = OrderedDict([
    ("mouth", (48, 68)),
    ("inner_mouth", (60, 68)),
    ("right_eyebrow", (17, 22)),
    ("left_eyebrow", (22, 27)),
    ("right_eye", (36, 42)),
    ("left_eye", (42, 48)),
    ("nose", (27, 36)),
    ("jaw", (0, 17))
])

def visualize_facial_landmarks(image, shape, color_code, alpha):
    # two copies of images, one for overlay and one for output
    overlay = image.copy()
    output = image.copy()

    # loop over the facial landmark regions individually
    for (i, name) in enumerate(FACIAL_LANDMARKS_IDXS.keys()):
    
        # check if are supposed to draw the line on lips
        if name == "mouth":
            # grab the coordinates associated with the mask
            (j, k) = FACIAL_LANDMARKS_IDXS[name]
            pts = shape[j:k]
            hull = cv2.convexHull(pts)
            cv2.drawContours(image=overlay, contours=[hull], contourIdx=0, color=color_code, thickness=-1)
            #cv2.drawContours(image=overlay, contours=[hull], contourIdx=1, color=color_code, lineType=4, thickness=-1)
        else:
            continue

    # apply the transparent overlay
    cv2.addWeighted(overlay, alpha, output, 1 - alpha, 0, output)

    # return the output image
    return output


def rgb_to_bgr(color):
    color[0], color[2] = color[2], color[0]
    return tuple([int(i) for i in color])

color_dict = {
    'red':[255,0,0],
    'blue':[0,0,255],
    'aqua':[0,255,255],
    'green4':[0,135,0],
    'green3':[0,215,0],
    'purple4':[95,0,135],
    'blue_violet':[95,0,255],
    'steel_blue1':[95,215,255],
    'yellow_4':[135,175,0],
    'dark_slate_gray':[135,255,255],
    'gold3':[175,175,0],
    'orchid':[215,95,215]
}

In [140]:
cap = cv2.VideoCapture(filename)

color_code = [178,34,34]
color_code = (34, 34, 178)
alpha = 0.5

if not cap.isOpened():
    print("Error in opening the video")

def tag_lips(frame, shape, color_code, alpha):
    overlay = frame.copy()
    output = frame.copy()
    
    pairs_indexes = [[50, 62, 61, 49], [51, 63, 62, 50], [52, 64, 63, 51], [52, 53, 54, 64],
                    [64, 54, 55, 65], [65, 55, 56, 66], [67, 66, 56, 57], [67, 66, 57, 58],
                    [68, 67, 58, 59], [61, 68, 59, 60], [49, 61, 68, 60]]
    
    for pair in pairs_indexes:
        coords = np.array( [[[shape[pair[0]-1][0], shape[pair[0]-1][1]], [shape[pair[1]-1][0], shape[pair[1]-1][1]], 
                [shape[pair[2]-1][0], shape[pair[2]-1][1]], [shape[pair[3]-1][0], shape[pair[3]-1][1]] ]])
        cv2.fillPoly(overlay, coords, color_code)
    
    #coords = np.array( [[[x_49, y_49], [x_50, y_50], [x_62, y_62], [x_61, y_61]]], dtype=np.int32)
    cv2.fillPoly(overlay, coords, color_code)
    cv2.addWeighted(overlay, alpha, output, 1-alpha, 0, output)
    return output

while True:
    ret, frame = cap.read()
    # resizing the screen to (360, 640) resolution
    frame = cv2.resize(frame, None, fx=0.5, fy=0.5, interpolation=cv2.INTER_AREA)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    rects = detector(gray, 1)
    for i, rect in enumerate(rects):
        shape = predictor(gray, rect)
        shape = face_utils.shape_to_np(shape)
        # shading only the lips area from the video
        #mouth_area = shape[48:68]
        for index, value in enumerate(shape):
            if index in range(48, 68):
                continue
            else:
                shape[index] = [0, 0]
        #output = visualize_facial_landmarks(frame, shape, color_code, alpha)
        output = tag_lips(frame, shape, color_code, alpha)
        cv2.imshow("Image", output)
    
    #cv2.imshow('Input', frame)

    c = cv2.waitKey(1)
    if c == 27:
        break

    

KeyboardInterrupt: 

In [141]:
cap.release()
cv2.destroyAllWindows()

In [26]:
import cv2
import numpy as np 
import base64
from io import BytesIO
from PIL import Image
import requests
import time
from ast import literal_eval

def process_frame(data):
    url = "http://127.0.0.1:5000/process/image/"
    payload = data
    headers = {
        'cache-control': "no-cache",
        'Postman-Token': "493db090-1235-4ee3-8b41-03014818e5c4"
        }
    response = requests.request("GET", url, data=payload, headers=headers)
    return response.text

# converting image to base64 string (encoding)
def image_to_base64(image):
    buff = BytesIO()
    image.save(buff, format='JPEG')
    image_string = base64.b64encode(buff.getvalue())
    return image_string

# converting base64 string to image (decoding)
def base64_to_image(data):
    buff = BytesIO(base64.b64decode(data))
    return Image.open(buff)
    
# convert PIL image to opencv image
def pil_to_opencv(image):
    return cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)

# convert opnecv image to pil
def opencv_to_pil(image):
    return Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

In [27]:
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    frame = opencv_to_pil(frame)
    frame = image_to_base64(frame)
    frame = process_frame(frame)
    frame = base64_to_image(frame)
    frame = pil_to_opencv(frame)
    frame = cv2.resize(frame, None, fx=0.5, fy=0.5, interpolation=cv2.INTER_AREA)
    cv2.imshow('Image', frame)
    
    c = cv2.waitKey(1)
    if c == 27:
        break
    

b'...'


b'...'


b'...'


b'...'


b'...'


...

b'...'


b'...'


b'...'


b'...'


b'...'


In [28]:
cap.release()
cv2.destroyAllWindows()